# Project: Data Modeling with Cassandra

## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
#print(len(full_data_rows_list))
# check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in event_datafile_new.csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Coding portion for data modeling with Cassandra

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of event_datafile_new.csv

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [2]:
# Create a connection to a Cassandra instance on the local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Create a session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [3]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### After setting up Cassandra, tables would be created to run queries
### There are 3 queries to be performed

### Query 1
#### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To answer the query, table session_songs is created

The selected attributes are sessionId, itemInSession, artist, songTitle and songLength

Among the selected attributes, sessionId and itemInSession are the primary key, where sessionId is the partition key and itemInSession is the clustering column, so as to fulfill the selection criteria

In [5]:
query = """
CREATE TABLE IF NOT EXISTS session_songs
(sessionId int,
itemInSession int,
artist text,
songTitle text,
songLength float,
PRIMARY KEY (sessionId, itemInSession))
"""

try:
    session.execute(query)
except Exception as e:
    print(e)             

Insert data to table session_songs from event_datafile_new.csv

In [6]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = """
        INSERT INTO session_songs
        (sessionId, itemInSession, artist, songTitle, songLength)
        VALUES (%s, %s, %s, %s, %s)
        """
    
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

To answer the query, column artist, songTitle and songLength are selected from table session_songs

Use the key sessionId to do the partition and itemInSession to do the clustering

In [7]:
query = """
SELECT artist, songTitle, songLength
FROM session_songs
WHERE sessionId = 338 AND itemInSession = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
result = pd.DataFrame(list(rows))
result

,artist,songtitle,songlength
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2
#### 2. Give name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer the query, table user_session_songs is created

The selected attributes are userId, sessionId, itemInSession, artist, song, user_firstName and user_lastName

Among the selected attributes, userId, sessionId and itemInSession are the primary key, where uerId and sessionId formed as composite and used as the partition key. itemInSession is the clustering column, so as to fulfill the selection criteria

In [8]:
query = """
CREATE TABLE IF NOT EXISTS user_session_songs
(userId int, 
sessionId int,
itemInSession int, 
artist text, 
song text, 
user_firstName text, 
user_lastName text,  
PRIMARY KEY ((userId, sessionId), itemInSession))
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

Insert data to table user_session_songs from event_datafile_new.csv

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = """
        INSERT INTO user_session_songs
        (userId, sessionId, itemInSession, artist, song, user_firstName, user_lastName)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

To answer the query, column artist, song, user_firstName and user_lastName are selected from table user_session_songs

Use the key userId and sessionId to do the partition and itemInSession to do the clustering in which the query would be order by itemInSession

In [11]:
query = """
SELECT artist, song, user_firstName, user_lastName
FROM user_session_songs
WHERE userId = 10 AND sessionId = 182
ORDER BY itemInSession
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
result = pd.DataFrame(list(rows))
result

,artist,song,user_firstname,user_lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3
#### 3. Give user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer the query, table song_listened_users is created

The selected attributes are song, userId, firstName and lastName

Among the selected attributes, song, userId are the primary key, where song is the partition key. userId is the clustering column, so as to fulfill the selection criteria

In [13]:
query = """
CREATE TABLE IF NOT EXISTS song_listened_users
(song text,
userId int,
firstName text, 
lastName text,
PRIMARY KEY (song, userId)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)     

Insert data to table song_listened_users from event_datafile_new.csv

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listened_users (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

To answer the query, column firstName and lastName are selected from table song_listened_users

Use the key song to do the partition 

In [15]:
## Add in the SELECT statement to verify the data was entered into the table

query = """
SELECT firstName, lastName
FROM song_listened_users
WHERE song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
result = pd.DataFrame(list(rows))
result

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [16]:
## Drop the table before closing out the sessions
query = "drop table session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_listened_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()